# Setup

In [ ]:
from dotenv import load_dotenv

_ = load_dotenv()

In [ ]:
from aocd import submit
from aocd.models import Puzzle

import pandas as pd
import itertools
from copy import deepcopy

In [ ]:
puzzle = Puzzle(year=2022, day=14)

In [ ]:
example_input, example_soln_a, example_soln_b = (
    puzzle.examples[0].input_data,
    *puzzle.examples[0].answers,
)
input = puzzle.input_data

# Part A

In [ ]:
def solution_a(input: str):
    data = input.split("\n")
    data = [[tuple(map(int, y.split(","))) for y in x.split(" -> ")] for x in data]

    class Cave:
        def __init__(self, data):
            rows = [
                min(list(itertools.chain(*[[y[0] for y in x] for x in data]))),
                max(list(itertools.chain(*[[y[0] for y in x] for x in data]))),
            ]
            cols = [
                min(list(itertools.chain(*[[y[1] for y in x] for x in data]))),
                max(list(itertools.chain(*[[y[1] for y in x] for x in data]))),
            ]
            self.rows = range(rows[0], rows[1] + 1)
            self.columns = range(0, cols[1] + 1)
            self.data = data
            self.df = self.cave_df_fill()
            self.full = False

        def cave_df_fill(self):
            cave_df = pd.DataFrame(index=self.rows, columns=self.columns)
            for path in self.data:
                for line in [(path[i], path[i + 1]) for i in range(len(path) - 1)]:
                    start, end = line
                    if start[0] == end[0]:
                        for i in range(
                            min(start[1], end[1]), max(start[1], end[1]) + 1
                        ):
                            cave_df.loc[start[0], i] = "#"
                    else:
                        for i in range(
                            min(start[0], end[0]), max(start[0], end[0]) + 1
                        ):
                            cave_df.loc[i, start[1]] = "#"
            cave_df.fillna(".", inplace=True)
            return cave_df.T

        def sand(self):
            unblocked = True
            sand = [500, 0]
            while unblocked and not self.full:
                try:
                    if self.df.loc[sand[1] + 1].at[sand[0]] == ".":
                        sand[1] += 1
                    elif self.df.loc[sand[1] + 1].at[sand[0] - 1] == ".":
                        sand[0] -= 1
                        sand[1] += 1
                    elif self.df.loc[sand[1] + 1].at[sand[0] + 1] == ".":
                        sand[0] += 1
                        sand[1] += 1
                    else:
                        unblocked = False
                        self.df.loc[sand[1]].at[sand[0]] = "o"
                except:
                    self.full = True

        def fill_sand(self):
            i = 0
            while not self.full:
                self.sand()
                i += 1
            return i - 1

    cave = Cave(data)
    return cave.fill_sand()

In [ ]:
print("Part A example solution:", solution_a(input=example_input))
print("Part A example answer:", example_soln_a)

In [ ]:
solution_a_output = solution_a(input=input)
print("Part A solution:", solution_a_output, "\n" + "-" * 60)
submit(solution_a_output, day=14, year=2022, part="a")

# Part B

In [ ]:
def solution_b(input: str):
    data = input.split("\n")
    data = [[tuple(map(int, y.split(","))) for y in x.split(" -> ")] for x in data]

    # part 2

    class Cave2:
        def __init__(self, data):
            rows = [
                min(list(itertools.chain(*[[y[0] for y in x] for x in data]))),
                max(list(itertools.chain(*[[y[0] for y in x] for x in data]))),
            ]
            cols = [
                min(list(itertools.chain(*[[y[1] for y in x] for x in data]))),
                max(list(itertools.chain(*[[y[1] for y in x] for x in data]))),
            ]
            self.rows = range(rows[0], rows[1] + 1)
            self.columns = range(0, cols[1] + 3)
            self.data = data
            self.df = self.cave_df_fill()
            self.full = False

        def cave_df_fill(self):
            cave_df = pd.DataFrame(index=self.rows, columns=self.columns)
            for path in [
                *self.data,
                [
                    (min(self.rows), list(self.columns)[-1]),
                    (max(self.rows), list(self.columns)[-1]),
                ],
            ]:
                for line in [(path[i], path[i + 1]) for i in range(len(path) - 1)]:
                    start, end = line
                    if start[0] == end[0]:
                        for i in range(
                            min(start[1], end[1]), max(start[1], end[1]) + 1
                        ):
                            cave_df.loc[start[0], i] = "#"
                    else:
                        for i in range(
                            min(start[0], end[0]), max(start[0], end[0]) + 1
                        ):
                            cave_df.loc[i, start[1]] = "#"

            cave_df.fillna(".", inplace=True)
            return cave_df.T

        def sand(self):
            unblocked = True
            sand = [500, 0]
            while unblocked and not self.full:
                if sand[0] - 1 not in self.df.columns:
                    self.df = deepcopy(
                        pd.concat(
                            [
                                self.df,
                                pd.DataFrame(
                                    data={
                                        sand[0]
                                        - 1: ["."] * (self.df.shape[0] - 1)
                                        + ["#"]
                                    }
                                ),
                            ],
                            axis=1,
                        )
                    )

                if sand[0] + 1 not in self.df.columns:
                    self.df = deepcopy(
                        pd.concat(
                            [
                                self.df,
                                pd.DataFrame(
                                    data={
                                        sand[0]
                                        + 1: ["."] * (self.df.shape[0] - 1)
                                        + ["#"]
                                    }
                                ),
                            ],
                            axis=1,
                        )
                    )

                if self.df.loc[sand[1] + 1].at[sand[0]] == ".":
                    sand[1] += 1
                elif self.df.loc[sand[1] + 1].at[sand[0] - 1] == ".":
                    sand[0] -= 1
                    sand[1] += 1
                elif self.df.loc[sand[1] + 1].at[sand[0] + 1] == ".":
                    sand[0] += 1
                    sand[1] += 1
                else:
                    self.df.loc[sand[1]].at[sand[0]] = "o"
                    if sand == [500, 0]:
                        self.full = True
                    unblocked = False

        def fill_sand(self):
            i = 0
            while not self.full:
                self.sand()
                i += 1
            return i

    cave2 = Cave2(data)
    return cave2.fill_sand()

In [ ]:
print("Part B example solution:", solution_b(example_input))
print("Part B example answer:", example_soln_b)

In [ ]:
solution_b_output = solution_b(input)
print("Part B solution:", solution_b_output, "\n" + "-" * 60)
submit(solution_b_output, day=14, year=2022, part="b")